In [1]:
#Python Implementation of the Hartree Fock Method
#Procedures listed in the code follow as described in Modern Quantum Chemistry: 
#Introduction to Advanced Electronic Structure Theory, By Attila Szabo and Neil S. Ostlund
import math
import sys
import numpy as np
from molecule import atom
from molecule import vector
from molecule import guassian
from molecule import molecule
from notebookImporter import importNotebook

#import integrals notebook for the hartree method
integrals = importNotebook("hartree_integrals")
scf = importNotebook("hartree_scf")

#define SCF convergence critera, and max number of iteration cycles
SCF_CONVERGENCE = pow(10, -100)
MAX_ITERATIONS = 100

#Step 1
#Specify Molecules, Nuclear Coordinates, and Charge of the nucli Number of Electrons,

#generate an h2 atom with a distance of 1.4 AU to compare with Szabo pg. 160
R = 1.4
system = molecule()
system.addAtom(atom(vector(1,1,1), 2, 2))
#system.addAtom(atom(vector(1,1,1+R), 1,1))

#add a basis set
system.addBasis("6-21G")

system.display()

Molecule
Basis Set: 6-21G
Total Number of Electrons: 2
Atoms: 
     Atomic Number: 2, Electrons: 2, Coordinate: X: 1 Y: 1 Z: 1


In [2]:
#Step 2
#Calculate Integrals
#Overlap, KE, Nuclear Attraaction, and electron repulsion
S = integrals.overlap(system)
print("Overlap Matrix: ")
print(np.matrix(S))
print()

T = integrals.kineticEnergy(system)
print("Electron Kinetic Energy Matrix: ")
print(np.matrix(T))
print()

V = integrals.nuclearAttraction(system)
for index, atom in enumerate(V):
    print("Nucli " + str(index) + "-Electron Attraction Matrix: ")
    print(np.matrix(atom))
print()

electronRepulsion = integrals.electronElectronRepulsion(system)
print("Electron Repulsion Tensor: ")
print(np.array(electronRepulsion))
print()

#Form the electronic hamiltonian
H = np.matrix(T)

#add in all of the nuclear attractions matricies to the hamiltonian
for atom in V:
    H += np.matrix(atom)
        
print("Electronic Hamiltonian :")
print(H)
print()

Overlap Matrix: 
[[1.         0.59521595]
 [0.59521595 1.        ]]

Electron Kinetic Energy Matrix: 
[[3.9161238 0.5789494]
 [0.5789494 0.5744895]]

Nucli 0-Electron Attraction Matrix: 
[[-5.48968847 -2.23529223]
 [-2.23529223 -1.97512902]]

Electron Repulsion Tensor: 
[[[[1.80325459 0.87650682]
   [0.87650682 0.91700828]]

  [[0.87650682 0.45207726]
   [0.45207726 0.51467807]]]


 [[[0.87650682 0.45207726]
   [0.45207726 0.51467807]]

  [[0.91700828 0.51467807]
   [0.51467807 0.69831356]]]]

Electronic Hamiltonian :
[[-1.57356468 -1.65634283]
 [-1.65634283 -1.40063952]]



In [3]:
#Prepare for the SCF procedure

#get size of the basis set 
size = len(S)

#compute the Transformation Matrix
X = scf.X(S, size)

#get guess Fock matrix, assume 2-electron term is equal to 0
F = H

In [4]:
# SCF Procedure 

#init list to store the energy from each iteration
#as well as a boolean to signify whether the loop has converged
E = []
converged = False

while( not converged ):
  
    #diagnolze the Fock matrix and convert it to MO basis 
    F = X.transpose() * F * X 
    
   #diagnolize the Fock Matrix to obtain the MOs and the their respective energies
    MOEnergy, MO = np.linalg.eigh(F)

    #Transform the MO basis MOs to an AO basis
    C = X * MO
    
    #compute the electron density, the two electron term, and then use G to compute the new Fock matrix
    P = scf.densityMatrix(C, system.N, size)
    G = scf.G(electronRepulsion, P, size)
    F = H + G
    
    #compute the new expectation energy 
    E.append(scf.expectationEnergy(H, F, P))
    
    #check if at least two SCF iterations have occured
    #if more than two have occured, then check if the difference betweeen this E, 
    #and the previous E is less then the covergence value, if yes, end the SCF loop
    #if energy has not converged, check whether the max number of iterations have occured so far
    sizeE = len(E)
    if(len(E) > 2):
        if(abs(E[sizeE-2] - E[sizeE-1]) < SCF_CONVERGENCE):
            converged = True
        elif(sizeE > MAX_ITERATIONS):
            print("SCF Failed to Converge")
            break
    
    #compute total energy of the system including nuclear-nuclear repulsion
    totalE = E[sizeE-1] + scf.nuclearRepulsion(system)
    
    #display information about current SCF iteration to the user
    print("SCF Iteration #" + str(sizeE) + ", Electronic Energy: " + str(E[sizeE-1]) + " Hartrees, Total Energy: " + str(totalE) + " Hartrees")

SCF Iteration #1, Electronic Energy: -2.7351099044716616 Hartrees, Total Energy: -2.7351099044716616 Hartrees
SCF Iteration #2, Electronic Energy: -2.835262969514034 Hartrees, Total Energy: -2.835262969514034 Hartrees
SCF Iteration #3, Electronic Energy: -2.835678080552964 Hartrees, Total Energy: -2.835678080552964 Hartrees
SCF Iteration #4, Electronic Energy: -2.8356798659098947 Hartrees, Total Energy: -2.8356798659098947 Hartrees
SCF Iteration #5, Electronic Energy: -2.835679873607232 Hartrees, Total Energy: -2.835679873607232 Hartrees
SCF Iteration #6, Electronic Energy: -2.835679873640423 Hartrees, Total Energy: -2.835679873640423 Hartrees
SCF Iteration #7, Electronic Energy: -2.8356798736405664 Hartrees, Total Energy: -2.8356798736405664 Hartrees
SCF Iteration #8, Electronic Energy: -2.835679873640567 Hartrees, Total Energy: -2.835679873640567 Hartrees
SCF Iteration #9, Electronic Energy: -2.8356798736405677 Hartrees, Total Energy: -2.8356798736405677 Hartrees
SCF Iteration #10, E